In [49]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from collections import Counter

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [50]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\non_linear\non_linear_fft_tests.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\non_linear\model_fft')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear
Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear\model_fft


In [51]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\non_linear\non_linear_fft_tests.pkl


In [52]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [53]:
def get_modes_base_and_coeficients(data, n_modes):

    # FFT decomposition and obtain energy of each mode
    fft_data = torch.fft.fft2(data)
    fft_data_shifted = torch.fft.fftshift(fft_data)
    energy = torch.abs(fft_data_shifted)
    energy_flattened = energy.flatten(1, 2)

    # Get the n_modes more energetic modes and their indices
    top_energetic = torch.topk(energy_flattened, n_modes).indices
    top_energetic_indices, _ = zip(*Counter(top_energetic.flatten()).most_common(n_modes))

    # Get base and coefficients
    modes_base = list(map(int, top_energetic_indices))
    coeficients = fft_data_shifted.flatten(1, 2)[:, top_energetic_indices]

    return coeficients, modes_base


def reconstruct_data_fft(coeficients, base, data_shape):

    # Create the new tensor to allocate filtered modes
    filtered_modes = torch.zeros((coeficients.shape[0], data_shape[1]*data_shape[2]), dtype=torch.complex64)
    filtered_modes[:, base] = coeficients

    # Reshape filtered modes and reconstruct image
    filtered_modes_base_fft_shifted =  filtered_modes.reshape(data_shape)
    filtered_modes_base = torch.fft.ifftshift(filtered_modes_base_fft_shifted)

    reconstructed_data = torch.real(torch.fft.ifft2(filtered_modes_base)) 

    return reconstructed_data

In [54]:
n_modes = 19

u_original = torch.Tensor(dataset['y_train'])
u_coeficients, u_modes_base = get_modes_base_and_coeficients(u_original, n_modes)
u_reconstructed = reconstruct_data_fft(u_coeficients, u_modes_base, u_original.shape)

real_part = u_coeficients.real
imag_part = u_coeficients.imag

y_train = torch.stack([real_part, imag_part], dim=-1)

In [55]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True)
f_train = torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True)

input_size = X_train[0].shape
predictive_output_size = y_train[0].shape
explanatory_output_size = K_train[0].shape

In [56]:
def reconstruct_data_fft_NN(coeficients, base, data_shape):

    # Create the new tensor to allocate filtered modes
    filtered_modes = torch.zeros((coeficients.shape[0], data_shape[0]*data_shape[1]), dtype=torch.complex64)
    filtered_modes[:, base] = coeficients

    # Reshape filtered modes and reconstruct image
    filtered_modes_base_fft_shifted =  filtered_modes.reshape((coeficients.shape[0], *data_shape))
    filtered_modes_base = torch.fft.ifftshift(filtered_modes_base_fft_shifted)

    reconstructed_data = torch.real(torch.fft.ifft2(filtered_modes_base)) 

    return reconstructed_data

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cpu


In [58]:
u_original = torch.Tensor(dataset['y_train'])
u_coeficients, u_modes_base = get_modes_base_and_coeficients(u_original, n_modes)
u_reconstructed = reconstruct_data_fft(u_coeficients, u_modes_base, u_original.shape)

real_part = u_coeficients.real
imag_part = u_coeficients.imag

y_train = torch.stack([real_part, imag_part], dim=-1)

In [59]:
u_original = torch.Tensor(dataset['y_val'])
u_coeficients, u_modes_base = get_modes_base_and_coeficients(u_original, n_modes)
u_reconstructed = reconstruct_data_fft(u_coeficients, u_modes_base, u_original.shape)

real_part = u_coeficients.real
imag_part = u_coeficients.imag

y_val = torch.stack([real_part, imag_part], dim=-1)

In [60]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
# y_train = TensOps(fourier_transform_split(dataset['y_train']).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
# y_val = TensOps(fourier_transform_split(dataset['y_val']).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train
K_np = K_train.values
f_np = f_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np, f_train_np, f_test_np = train_test_split(X_np, y_np, K_np, f_np, test_size=0.2, random_state=42)

X_train = X_train_np.to(device)
X_test = X_test_np.to(device)

y_train = TensOps(y_train_np.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test_np.requires_grad_(True).to(device), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

f_train = TensOps(f_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
f_test = TensOps(f_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [61]:
# Se carga el modelo y el optimizador
model = FFTNeuralNetwork(input_size, predictive_output_size, explanatory_output_size, modes_base=u_modes_base)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 2000

batch_size = 64
n_checkpoints = 100

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )

Start training
Epoch 0, Train loss: 1.019e+12, Test loss: 1.002e+12, MSE(e): 1.019e+04, MSE(pi1): 9.006e+00, MSE(pi2): 4.216e+01, MSE(pi3): 2.606e-01
Epoch 100, Train loss: 9.515e+11, Test loss: 9.336e+11, MSE(e): 9.515e+03, MSE(pi1): 1.356e+00, MSE(pi2): 4.364e+01, MSE(pi3): 2.013e-01
Epoch 200, Train loss: 8.922e+11, Test loss: 8.744e+11, MSE(e): 8.922e+03, MSE(pi1): 6.323e-01, MSE(pi2): 4.203e+01, MSE(pi3): 1.746e-01
Epoch 300, Train loss: 8.287e+11, Test loss: 8.118e+11, MSE(e): 8.287e+03, MSE(pi1): 6.523e-01, MSE(pi2): 3.921e+01, MSE(pi3): 1.789e-01
Epoch 400, Train loss: 7.660e+11, Test loss: 7.500e+11, MSE(e): 7.660e+03, MSE(pi1): 6.531e-01, MSE(pi2): 3.650e+01, MSE(pi3): 1.791e-01
Epoch 500, Train loss: 7.085e+11, Test loss: 6.934e+11, MSE(e): 7.085e+03, MSE(pi1): 6.527e-01, MSE(pi2): 3.397e+01, MSE(pi3): 1.791e-01
Epoch 600, Train loss: 6.564e+11, Test loss: 6.422e+11, MSE(e): 6.564e+03, MSE(pi1): 6.523e-01, MSE(pi2): 3.168e+01, MSE(pi3): 1.791e-01
Epoch 700, Train loss: 6.071

KeyboardInterrupt: 